# Outline<a class = "anchor" id = 'outline'>
   [1. Background](#background)<br>
   
   [2. Data Section](#data)<br>
   - [Step 1: Import Package Dependencies](#package)<br>
   - [Step 2: Import Data](#import)<br>
   - [Step 3: Clean Data](#clean)<br>
   - [Step 4: Train/Test Split](#split)<br>
   
   [3. Classification Model](#model)<br>
   - [Step 1: Logistic Regression](#log_reg)<br>
   - [Step 2: Random Forest](#random)<br>
   - [Step 3: XGBoost](#XGB)<br>
   - [Step 4: kNN](#knn)<br>
   
   [4. Summary](#sum)

# Background<a class = "anchor" id = 'background'>
    
[Go to top](#outline)

- 119,390 hotel records where each record represents a single booking: hotel_bookings.csv<br>
- Target Variable: ‘is_canceled’<br>
- Detailed descriptions of all variables can be found at https://www.kaggle.com/sanjana08/hotel-booking-cancellation-prediction/dataLinks.

# Data Section<a class = "anchor" id = 'data'>
    
[Go to top](#outline)

> **Step 1:** Import Package Dependencies<br>
    
> **Step 2:** Import Data
  
> **Step 3:** Clean Data
    
> **Step 4:** Train/Test Split

## Step 1: Import Package Dependencies<a class = "anchor" id = 'package'>

In [23]:
import warnings
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_score, classification_report

## Step 2: Import Data<a class = "anchor" id = 'import'>

In [3]:
df = pd.read_csv("hotel_bookings.csv")

In [4]:
df.info() # contains 119390 rows of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

## Step 3: Clean Data<a class = "anchor" id = 'clean'>
- Clean duplicate data
- Encode categorical columns
- Replace null value
- Build a DataFrame of Correlation Coefficient
- Bulid a new DataFrame from the colomns whose correlation coeffcient is bigger than 0.1
- Remove the outlier

### Clean duplicate data

In [5]:
# Clean duplicate data
df = df.drop_duplicates()

In [6]:
df.info() # 87138 rows of data remain

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87138 entries, 0 to 119389
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87138 non-null  object 
 1   is_canceled                     87138 non-null  int64  
 2   lead_time                       87138 non-null  int64  
 3   arrival_date_year               87138 non-null  int64  
 4   arrival_date_month              87138 non-null  object 
 5   arrival_date_week_number        87138 non-null  int64  
 6   arrival_date_day_of_month       87138 non-null  int64  
 7   stays_in_weekend_nights         87138 non-null  int64  
 8   stays_in_week_nights            87138 non-null  int64  
 9   adults                          87138 non-null  int64  
 10  children                        87134 non-null  float64
 11  babies                          87138 non-null  int64  
 12  meal                           

### Encode categorical columns

In [7]:
columns = df.columns

In [8]:
# Encode categorical columns
le = LabelEncoder()

for i in columns:
    if df[i].dtype == object:
        df[i] = le.fit_transform(df[i])

In [9]:
df.info() # every dtype turn into int/float

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87138 entries, 0 to 119389
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87138 non-null  int64  
 1   is_canceled                     87138 non-null  int64  
 2   lead_time                       87138 non-null  int64  
 3   arrival_date_year               87138 non-null  int64  
 4   arrival_date_month              87138 non-null  int64  
 5   arrival_date_week_number        87138 non-null  int64  
 6   arrival_date_day_of_month       87138 non-null  int64  
 7   stays_in_weekend_nights         87138 non-null  int64  
 8   stays_in_week_nights            87138 non-null  int64  
 9   adults                          87138 non-null  int64  
 10  children                        87134 non-null  float64
 11  babies                          87138 non-null  int64  
 12  meal                           

### Replace null value

In [10]:
df = df.fillna(0)

In [11]:
df.info() # Non-Null count are 87138

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87138 entries, 0 to 119389
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87138 non-null  int64  
 1   is_canceled                     87138 non-null  int64  
 2   lead_time                       87138 non-null  int64  
 3   arrival_date_year               87138 non-null  int64  
 4   arrival_date_month              87138 non-null  int64  
 5   arrival_date_week_number        87138 non-null  int64  
 6   arrival_date_day_of_month       87138 non-null  int64  
 7   stays_in_weekend_nights         87138 non-null  int64  
 8   stays_in_week_nights            87138 non-null  int64  
 9   adults                          87138 non-null  int64  
 10  children                        87138 non-null  float64
 11  babies                          87138 non-null  int64  
 12  meal                           

### Build a DataFrame of Higher Correlation Coefficient

In [12]:
# Correlation Coefficient
correlations = df.corr()['is_canceled']

In [13]:
correlations

hotel                            -0.071313
is_canceled                       1.000000
lead_time                         0.182786
arrival_date_year                 0.089073
arrival_date_month               -0.045481
arrival_date_week_number          0.001216
arrival_date_day_of_month         0.005147
stays_in_weekend_nights           0.060510
stays_in_week_nights              0.083423
adults                            0.082338
children                          0.068455
babies                           -0.020164
meal                              0.045368
country                           0.088905
market_segment                    0.183234
distribution_channel              0.150163
is_repeated_guest                -0.089136
previous_cancellations            0.050900
previous_bookings_not_canceled   -0.051847
reserved_room_type                0.049911
assigned_room_type               -0.060151
booking_changes                  -0.092695
deposit_type                      0.136538
agent      

In [14]:
col = [] # columns
cor = [] # correlations
for i in correlations:
    cor.append(i)
for j in columns:
    col.append(j)
dict_cor = dict(zip(col, cor))
df_cor = pd.DataFrame(list(dict_cor.items()), columns=['Column', 'Coeff'])

In [15]:
df_cor

,Column,Coeff
0,hotel,-0.071313
1,is_canceled,1.000000
2,lead_time,0.182786
3,arrival_date_year,0.089073
4,arrival_date_month,-0.045481
5,arrival_date_week_number,0.001216
6,arrival_date_day_of_month,0.005147
7,stays_in_weekend_nights,0.060510
8,stays_in_week_nights,0.083423
9,adults,0.082338


### Bulid a new DataFrame

In [16]:
# Bulid a new DataFrame from the colomns whose correlation coeffcient is bigger than 0.1
big_cor_columns = df_cor[df_cor['Coeff'].abs() >= 0.1]
new_columns = list(big_cor_columns['Column'])
df = df[new_columns]

In [17]:
df.info() # 8 columns remain

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87138 entries, 0 to 119389
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   is_canceled                  87138 non-null  int64  
 1   lead_time                    87138 non-null  int64  
 2   market_segment               87138 non-null  int64  
 3   distribution_channel         87138 non-null  int64  
 4   deposit_type                 87138 non-null  int64  
 5   adr                          87138 non-null  float64
 6   required_car_parking_spaces  87138 non-null  int64  
 7   total_of_special_requests    87138 non-null  int64  
dtypes: float64(1), int64(7)
memory usage: 6.0 MB


### Remove the outliers

In [18]:
def remove_outliers(df, column, threshold=1.5):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [19]:
for i in df.columns:
    df = remove_outliers(df, i)

In [20]:
df.info() # 58105 rows of data remain

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58105 entries, 4 to 119389
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   is_canceled                  58105 non-null  int64  
 1   lead_time                    58105 non-null  int64  
 2   market_segment               58105 non-null  int64  
 3   distribution_channel         58105 non-null  int64  
 4   deposit_type                 58105 non-null  int64  
 5   adr                          58105 non-null  float64
 6   required_car_parking_spaces  58105 non-null  int64  
 7   total_of_special_requests    58105 non-null  int64  
dtypes: float64(1), int64(7)
memory usage: 4.0 MB


## Step 4: Train/Test Split<a class="anchor" id="split">

In [21]:
X = df.drop(columns=['is_canceled'])
y = df['is_canceled']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# Classification Model<a class = "anchor" id = 'model'>
    
[Go to top](#outline)

> **Step 1:** Logistic Regression<br>
    
> **Step 2:** Random Forest<br>
 
> **Step 3:** XGBoost<br>
    
> **Step 4:** kNN

## Logistic Regression<a class = "anchor" id = 'log_reg'>

In [439]:
# Create a logistic regression model
logistic_model = LogisticRegression()

# Train the model on the training data
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [441]:
print(f'Accuracy: {accuracy}')
print(f'f1 score: {f1}')
print(" ")
print(classification_rep)

Accuracy: 0.7282010096374484
f1 score: 0.4291566265060241
 
              precision    recall  f1-score   support

           0       0.74      0.92      0.82     11878
           1       0.65      0.32      0.43      5554

    accuracy                           0.73     17432
   macro avg       0.70      0.62      0.63     17432
weighted avg       0.71      0.73      0.70     17432



## Random Forest<a class = "anchor" id = 'random'>
1. Number of Trees (n_estimators):<br>
    - Determines the number of trees in the forest.<br>
    - Generally, more trees increase the model's performance and robustness but also increase computational time and complexity.<br>
 
2. Maximum Depth of Trees (max_depth):<br>
    - Defines the maximum depth of each tree.<br>
    - Deeper trees can model more complex patterns but may lead to overfitting.<br>

3. Minimum Samples for Split (min_samples_split):<br>
    - The minimum number of samples required to split an internal node.<br>
    - Higher values prevent creating nodes that only fit to noise in the data.<br>

4. Minimum Samples for Leaf (min_samples_leaf):<br>
    - The minimum number of samples required to be at a leaf node.<br>
    - Useful to control overfitting by smoothing the model, especially for regression.<br>

5. Maximum Features (max_features):<br>
    - The number of features to consider when looking for the best split.<br>
    - Choices include 'auto', 'sqrt', 'log2', or a fraction of the total features.<br>

In [442]:
# Custom scorers for evaluation
scorers = {
    'precision': make_scorer(precision_score, pos_label=1),
    'recall': make_scorer(recall_score, pos_label=1),
    'f1': make_scorer(f1_score, pos_label=1),
    'accuracy': make_scorer(accuracy_score),
    'auc': make_scorer(roc_auc_score)
}

# Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 200],         # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],        # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],        # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],          # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt'],       # The number of features to consider when looking for the best split
    'class_weight': [None, 'balanced']      # Weights associated with classes in the form {class_label: weight}
}

# Grid Search for hyperparameter tuning with custom scorers
grid_search_rf = GridSearchCV(estimator=rf_classifier,
                              param_grid=param_grid,
                              scoring=scorers,
                              refit='f1',  # Refitting on the basis of F1 score, you can choose any other metric
                              cv=5,
                              verbose=2,
                              n_jobs=-1)

# Fitting the model
grid_search_rf.fit(X_train, y_train)

# Extract the best parameters
best_rf_params = grid_search_rf.best_params_

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_sampl

/Users/homerliu/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END class_w

[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END class

[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END class_weight=None, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END class_weight=N

[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   8.8s
[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   4.3s
[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=50; total time=   2.2s
[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=50; total time=   2.4s
[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=50; total time=   2.0s
[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   3.9s
[CV] END class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   2.1s
[CV] END class_weight=None, ma

[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   1.9s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   2.2s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   2.2s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   1.9s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   2.0s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   3.8s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   3.6s
[CV] END class_wei

[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   3.2s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   3.4s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   3.7s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   3.5s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   8.4s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   7.5s
[CV] END class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   7.6s
[CV] E

/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
1080 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
549 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 637, in _validate_params
    validate_parameter_constraints

In [443]:
best_rf_params

{'class_weight': 'balanced',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 100}

In [24]:
best_params = {'class_weight': 'balanced',
               'max_depth': 10,
               'max_features': 'sqrt',
               'min_samples_leaf': 2,
               'min_samples_split': 5,
               'n_estimators': 100}

# Initialize the Random Forest model with the best parameters
rf_model = RandomForestClassifier(**best_params, random_state=42)

start_time = time.time()
rf_model.fit(X_train, y_train)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

Training time: 1.603111743927002 seconds


In [445]:
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.7122533272143184
F1 Score: 0.5969789490599389


## XGBoost<a class = "anchor" id = 'XGB'>
    
1. Start tuning with high-impact parameters
- n_estimators
- max_depth
- learning_rate
2. then refine the model by adjusting the other parameters
- min_child_weight
- min_samples_split
- min_samples_leaf
- max_features
- scale_pos_weight
- alpha

In [446]:
# Custom scorers
scorers = {
    'precision': make_scorer(precision_score, pos_label=1),
    'recall': make_scorer(recall_score, pos_label=1),
    'f1': make_scorer(f1_score, pos_label=1),
    'accuracy': make_scorer(accuracy_score),
    'auc': make_scorer(roc_auc_score)
}

# XGBoost Classifier
xgb_classifier = XGBClassifier(random_state=42)

# Hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 200], # Number of boosting rounds, default=100
    'max_depth': [3, 5, 7], # default=6
    'learning_rate': [0.01, 0.1, 0.2] # default=0.3
}

# Grid Search with multiple scorers
grid_search = GridSearchCV(estimator=xgb_classifier,
                           param_grid=param_grid,
                           scoring=scorers,
                           refit='f1',  # Can change to any key in scorers dictionary
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

# Fitting the model
grid_search.fit(X_train, y_train)

# Accessing the results
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results

Fitting 5 folds for each of 27 candidates, totalling 135 fits


/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/homerliu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classifica

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_precision,split1_test_precision,...,std_test_accuracy,rank_test_accuracy,split0_test_auc,split1_test_auc,split2_test_auc,split3_test_auc,split4_test_auc,mean_test_auc,std_test_auc,rank_test_auc
0,0.100422,0.018009,0.018557,0.005368,0.01,3,50,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.000000,0.000000,...,0.000040,26,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,26
1,0.210573,0.030020,0.023232,0.007258,0.01,3,100,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.630507,0.637901,...,0.004192,21,0.662670,0.657231,0.662634,0.657813,0.656128,0.659295,0.002794,22
2,0.380476,0.066390,0.037593,0.011219,0.01,3,200,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.634026,0.637143,...,0.004853,20,0.665576,0.660063,0.672641,0.658171,0.660143,0.663319,0.005276,21
3,0.146090,0.015894,0.027522,0.003309,0.01,5,50,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.000000,0.000000,...,0.000040,26,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,26
4,0.314638,0.069457,0.037036,0.009646,0.01,5,100,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.649611,0.667444,...,0.005336,23,0.646698,0.628810,0.648039,0.637210,0.640458,0.640243,0.006963,23
5,0.566791,0.033035,0.042852,0.010334,0.01,5,200,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.639483,0.646586,...,0.005455,15,0.663807,0.664105,0.676460,0.658348,0.657131,0.663970,0.006847,20
6,0.255107,0.053915,0.032360,0.008626,0.01,7,50,"{'learning_rate': 0.01, 'max_depth': 7, 'n_est...",1.000000,0.875000,...,0.000663,25,0.501947,0.501273,0.502277,0.505035,0.502576,0.502622,0.001282,25
7,0.431908,0.050902,0.038100,0.008453,0.01,7,100,"{'learning_rate': 0.01, 'max_depth': 7, 'n_est...",0.667134,0.669732,...,0.003768,24,0.642696,0.631461,0.633630,0.628244,0.621599,0.631526,0.006907,24
8,0.810779,0.072555,0.062842,0.012627,0.01,7,200,"{'learning_rate': 0.01, 'max_depth': 7, 'n_est...",0.641615,0.644722,...,0.004300,7,0.665350,0.664960,0.673104,0.662722,0.660606,0.665348,0.004235,19
9,0.130085,0.021584,0.023123,0.006936,0.1,3,50,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.637351,0.637486,...,0.005364,13,0.668586,0.663163,0.678512,0.662726,0.660891,0.666776,0.006405,17


In [447]:
grid_search_results.sort_values(by='rank_test_f1')[['mean_test_f1','std_test_f1','params']][:10]

,mean_test_f1,std_test_f1,params
24,0.536003,0.005783,"{'learning_rate': 0.2, 'max_depth': 7, 'n_esti..."
17,0.535525,0.008319,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti..."
16,0.535125,0.006466,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti..."
25,0.535111,0.006424,"{'learning_rate': 0.2, 'max_depth': 7, 'n_esti..."
26,0.534077,0.004255,"{'learning_rate': 0.2, 'max_depth': 7, 'n_esti..."
20,0.534040,0.004709,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti..."
22,0.533638,0.006561,"{'learning_rate': 0.2, 'max_depth': 5, 'n_esti..."
15,0.533205,0.006028,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti..."
14,0.532269,0.004730,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti..."
23,0.532030,0.005926,"{'learning_rate': 0.2, 'max_depth': 5, 'n_esti..."


In [448]:
grid_search_results.iloc[24]['params']

{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 50}

{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 50}

In [449]:
# Fixed parameters
fixed_params = {
    'n_estimators': 50,
    'max_depth': 7,
    'learning_rate': 0.2
}

# Hyperparameters to tune
param_grid = {
    'min_child_weight': [1, 5, 10], # default=1
    'min_samples_split': [2, 5, 10], # default=2
    'min_samples_leaf': [1, 2, 4], # default=1
    'max_features': ['auto', 'sqrt'], # default='auto'
    'scale_pos_weight': [1, 10, 25], # default=1
    'alpha': [0, 0.1, 0.5, 1] # default=0
}

# Updating XGBoost Classifier with fixed parameters
xgb_classifier_fixed = XGBClassifier(**fixed_params, random_state=42)

# Grid Search for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_classifier_fixed,
                           param_grid=param_grid,
                           scoring='f1',  # focusing on F1 score for imbalanced classification
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

# Fitting the model
grid_search.fit(X_train, y_train)

# Extract the best parameters
best_xgb_params = grid_search.best_params_
print("Best Parameters:", best_xgb_params)


Fitting 5 folds for each of 648 candidates, totalling 3240 fits


/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:25] WARNING: /Users/runner/work/xgboost/xgb

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:26] WARNING: /Users/runner/work/xgboost/xgb

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:27] WARNING: /Users/runner/work/xgboost/xgb

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:08:28] WARNING: /Users/runner/work/xgboost/xgb

...

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:52] WARNING: /Users/runner/work/xgboost/xgb

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:53] WARNING: /Users/runner/work/xgboost/xgb

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:54] WARNING: /Users/runner/work/xgboost/xgb

Best Parameters: {'alpha': 0.1, 'max_features': 'auto', 'min_child_weight': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'scale_pos_weight': 10}


/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:09:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [451]:
pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score')[['mean_test_score','std_test_score','params']][:10]

,mean_test_score,std_test_score,params
265,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'sqrt', 'min_ch..."
169,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
175,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
166,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
178,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
181,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
172,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
184,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
187,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."
163,0.538203,0.002421,"{'alpha': 0.1, 'max_features': 'auto', 'min_ch..."


In [452]:
pd.DataFrame(grid_search.cv_results_).params[265]

{'alpha': 0.1,
 'max_features': 'sqrt',
 'min_child_weight': 1,
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'scale_pos_weight': 10}

In [453]:
best_params = {'min_child_weight': 1,
               'min_samples_split': 5,
               'min_samples_leaf': 4,
               'max_features': 'sqrt',
               'scale_pos_weight': 10,
               'alpha': 0.1,
               'n_estimators': 50,
               'max_depth': 7,
               'learning_rate': 0.2}

xgb_model = XGBClassifier(**best_params, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

/Users/homerliu/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:10:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [454]:
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.48313446535107846
F1 Score: 0.5371892336141361


## kNN<a class = "anchor" id = 'knn'>

In [455]:
scaler = StandardScaler()

# Fit and transform the training data
X_train = scaler.fit_transform(X_train)

# Transform the testing data
X_test = scaler.transform(X_test)

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)  # You can choose a different number for n_neighbors

# Fit the classifier to the data
knn.fit(X_train, y_train)

# Predicting the Test set results
y_pred = knn.predict(X_test)

# Evaluating the Algorithm
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7097292335933915
0.49910908730944364
              precision    recall  f1-score   support

           0       0.76      0.83      0.80     11878
           1       0.55      0.45      0.50      5554

    accuracy                           0.71     17432
   macro avg       0.66      0.64      0.65     17432
weighted avg       0.70      0.71      0.70     17432



# Summary<a class = "anchor" id = 'sum'>
    
[Go to top](#outline)

## Short List

In [461]:
result = {
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', 'kNN'],
    'Accuracy': [0.728, 0.712, 0.48, 0.71],
    'F1 Score': [0.429, 0.597, 0.537, 0.499]
}

df_result = pd.DataFrame(result)

In [462]:
df_result

,Model,Accuracy,F1 Score
0,Logistic Regression,0.728,0.429
1,Random Forest,0.712,0.597
2,XGBoost,0.480,0.537
3,kNN,0.710,0.499


## Description: Winning Model

The besr model in this analysis is `Random Forest`. 

The F1 score of RF mofel is the highest and the Accuracy is in the second place of all.<br>
Additionally, the Accuracy of RF is similar with the Accuaracy of Logistic Regression Model, which has the highest Accuracy of all.

The following part is the best parameter of Random Forest Model: 
- `class_weight`: 'balanced'
- `max_depth`: 10
- `max_features`: 'sqrt'
- `min_samples_leaf`: 2
- `min_samples_split`: 5
- `n_estimators`: 100

Training time is **1.6 seconds**.